# Project 1

**Dataset:** Speed dating


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import scipy.stats as stats
import pandas as pd


ModuleNotFoundError: No module named 'seaborn'

In [3]:
import wandb

super_secret_API_key = "d1678741bcd8eb1d90ca26aa8f2bf079256ac391"

wandb.login(key=super_secret_API_key)
# ak to chceme pouzivat, tak si tu budes musiet spravit ucet aby som ti mohol dat pristup
# https://wandb.ai/bizon-mimi-stu/ZNEUS_project1/workspace?nw=nwuserbizonmimi

C:\Users\andre\PycharmProjects\ZNEUS_2025\ZNEUS_1\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
C:\Users\andre\PycharmProjects\ZNEUS_2025\ZNEUS_1\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This m

True

In [11]:
# wandb test
import random

# Start a new wandb run to track this script.
run = wandb.init(
    entity="bizon-mimi-stu",  # team name
    project="ZNEUS_project1",  # project name
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 0.02,
        "architecture": "CNN",
        "dataset": "CIFAR-100",
        "epochs": 10,
    },
)

# Simulate training.
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset

    # Log metrics to wandb.
    run.log({"acc": acc, "loss": loss})

# Finish the run and upload any remaining data.
run.finish()

acc,▁▄▇▆█▇▇█
loss,█▃▃▃▂▁▂▁
acc,0.79636
loss,0.1948


In [169]:
df_dating = pd.read_csv("DATA/speeddating.csv")
df_dating.head(10)

C:\Users\bizon\AppData\Local\Temp\ipykernel_29056\4076172677.py:1: DtypeWarning: Columns (3,10,11,15,16,17,18,19,39,40,41,42,43,44,51,52,53,54,55,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,107,109) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dating = pd.read_csv("DATA/speeddating.csv")


,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,importance_same_race,importance_same_religion,d_importance_same_race,d_importance_same_religion,field,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,d_pref_o_attractive,d_pref_o_sincere,d_pref_o_intelligence,d_pref_o_funny,d_pref_o_ambitious,d_pref_o_shared_interests,attractive_o,sinsere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,d_attractive_o,d_sinsere_o,d_intelligence_o,d_funny_o,d_ambitous_o,d_shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,d_attractive_important,d_sincere_important,d_intellicence_important,d_funny_important,d_ambtition_important,d_shared_interests_important,attractive,sincere,intelligence,funny,ambition,d_attractive,d_sincere,d_intelligence,d_funny,d_ambition,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,d_attractive_partner,d_sincere_partner,d_intelligence_partner,d_funny_partner,d_ambition_partner,d_shared_interests_partner,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,d_sports,d_tvsports,d_exercise,d_dining,d_museums,d_art,d_hiking,d_gaming,d_clubbing,d_reading,d_tv,d_theater,d_movies,d_concerts,d_music,d_shopping,d_yoga,interests_correlate,d_interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,expected_num_matches,d_expected_happy_with_sd_people,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,0,1,female,21,27,6,[4-6],'Asian/Pacific Islander/Asian-American',European/Caucasian-American,0,2,4,[2-5],[2-5],Law,35,20,20,20,0,5,[21-100],[16-20],[16-20],[16-20],[0-15],[0-15],6,8,8,8,8,6,[6-8],[6-8],[6-8],[6-8],[6-8],[6-8],15,20,20,15,15,15,[0-15],[16-20],[16-20],[0-15],[0-15],[0-15],6,8,8,8,7,[6-8],[6-8],[6-8],[6-8],[6-8],6,9,7,7,6,5,[6-8],[9-10],[6-8],[6-8],[6-8],[0-5],9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,[9-10],[0-5],[6-8],[9-10],[0-5],[0-5],[0-5],[0-5],[0-5],[6-8],[9-10],[0-5],[9-10],[9-10],[9-10],[6-8],[0-5],0.14,[0-0.33],3,2,4,[0-4],[0-3],[3-5],7,6,[6-8],[5-6],0,1,0,0
1,0,1,female,21,22,1,[0-1],'Asian/Pacific Islander/Asian-American',European/Caucasian-American,0,2,4,[2-5],[2-5],Law,60,0,0,40,0,0,[21-100],[0-15],[0-15],[21-100],[0-15],[0-15],7,8,10,7,7,5,[6-8],[6-8],[9-10],[6-8],[6-8],[0-5],15,20,20,15,15,15,[0-15],[16-20],[16-20],[0-15],[0-15],[0-15],6,8,8,8,7,[6-8],[6-8],[6-8],[6-8],[6-8],7,8,7,8,5,6,[6-8],[6-8],[6-8],[6-8],[0-5],[6-8],9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,[9-10],[0-5],[6-8],[9-10],[0-5],[0-5],[0-5],[0-5],[0-5],[6-8],[9-10],[0-5],[9-10],[9-10],[9-10],[6-8],[0-5],0.54,[0.33-1],3,2,4,[0-4],[0-3],[3-5],7,5,[6-8],[5-6],1,1,0,0
2,1,1,female,21,22,1,[0-1],'Asian/Pacific Islander/Asian-American','Asian/Pacific Islander/Asian-American',1,2,4,[2-5],[2-5],Law,19,18,19,18,14,12,[16-20],[16-20],[16-20],[16-20],[0-15],[0-15],10,10,10,10,10,10,[9-10],[9-10],[9-10],[9-10],[9-10],[9-10],15,20,20,15,15,15,[0-15],[16-20],[16-20],[0-15],[0-15],[0-15],6,8,8,8,7,[6-8],[6-8],[6-8],[6-8],[6-8],5,8,9,8,5,7,[0-5],[6-8],[9-10],[6-8],[0-5],[6-8],9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,[9-10],[0-5],[6-8],[9-10],[0-5],[0-5],[0-5],[0-5],[0-5],[6-8],[9-10],[0-5],[9-10],[9-10],[9-10],[6-8],[0-5],0.16,[0-0.33],3,2,4,[0-4],[0-3],[3-5],7,?,[6-8],[0-4],1,1,1,1
3,0,1,female,21,23,2,[2-3],'Asian/Pacific Islander/Asian-American',European/Caucasian-American,0,2,4,[2-5],[2-5],Law,30,5,15,40,5,5,[21-100],[0-15],[0-15],[21-100],[0-15],[0-15],7,8,9,8,9,8,[6-8],[6-8],[9-10],[6-8],[9-10],[6-8],15,20,20,15,15,15,[0-15],[16-20],[16-20],[0-15],[0-15],[0-15],6,8,8,8,7,[6-8],[6-8],[6-8],[6-8],[6-8],7,6,8,7,6,8,[6-8],[6-8],[6-8],[6-8],[6-8],[6-8],9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,[9-10],[0-5],[6-8],[9-10],[0-5],[0-5],[0-5],[0-5],[0-5],[6-8],[9-10],[0-5],[9-10]

In [170]:
df_dating.replace("?", np.nan, inplace=True)

In [171]:
df_dating.shape

(8378, 123)

In [172]:
df_dating.info

<bound method DataFrame.info of       has_null  wave  gender age age_o  d_age d_d_age  \
0            0     1  female  21    27      6   [4-6]   
1            0     1  female  21    22      1   [0-1]   
2            1     1  female  21    22      1   [0-1]   
3            0     1  female  21    23      2   [2-3]   
4            0     1  female  21    24      3   [2-3]   
...        ...   ...     ...  ..   ...    ...     ...   
8373         1    21    male  25    26      1   [0-1]   
8374         1    21    male  25    24      1   [0-1]   
8375         1    21    male  25    29      4   [4-6]   
8376         1    21    male  25    22      3   [2-3]   
8377         1    21    male  25    22      3   [2-3]   

                                         race  \
0     'Asian/Pacific Islander/Asian-American'   
1     'Asian/Pacific Islander/Asian-American'   
2     'Asian/Pacific Islander/Asian-American'   
3     'Asian/Pacific Islander/Asian-American'   
4     'Asian/Pacific Islander/Asian-Am

In [173]:
df_dating.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 123 columns):
 #    Column                           Dtype 
---   ------                           ----- 
 0    has_null                         int64 
 1    wave                             int64 
 2    gender                           object
 3    age                              object
 4    age_o                            object
 5    d_age                            int64 
 6    d_d_age                          object
 7    race                             object
 8    race_o                           object
 9    samerace                         int64 
 10   importance_same_race             object
 11   importance_same_religion         object
 12   d_importance_same_race           object
 13   d_importance_same_religion       object
 14   field                            object
 15   pref_o_attractive                object
 16   pref_o_sincere                   object
 17   pref_o_intel

In [174]:
missing_values = df_dating.isnull().sum().sort_values(ascending=False)

# Percentage of missing values
missing_percentage = (df_dating.isnull().mean() * 100).sort_values(ascending=False)

missing_df = pd.DataFrame(
    {"Missing Values": missing_values, "Percentage": missing_percentage}
)

print(missing_df.head(50))

                               Missing Values  Percentage
expected_num_interested_in_me            6578   78.515159
expected_num_matches                     1173   14.000955
shared_interests_o                       1076   12.843161
shared_interests_partner                 1067   12.735736
ambitous_o                                722    8.617809
ambition_partner                          712    8.498448
met                                       375    4.476009
funny_o                                   360    4.296968
funny_partner                             350    4.177608
guess_prob_liked                          309    3.688231
intelligence_o                            306    3.652423
intelligence_partner                      296    3.533063
sinsere_o                                 287    3.425639
sincere_partner                           277    3.306278
like                                      240    2.864646
attractive_o                              212    2.530437
attractive_par

In [175]:
df_dating = df_dating.drop(columns="expected_num_interested_in_me")

In [176]:
duplicates = df_dating.duplicated().sum()

print(f"Number of duplicate rows: {duplicates}")


# df = df.drop_duplicates()

Number of duplicate rows: 0


In [177]:
df_dating.dtypes.value_counts()  # How many numeric, object, etc.

categorical = df_dating.select_dtypes(include=["object"]).columns
for col in categorical[:10]:  # Show first 10 to avoid spam
    print(f"{col}: {df_dating[col].nunique()} unique values")

gender: 2 unique values
age: 30 unique values
age_o: 24 unique values
d_d_age: 4 unique values
race: 5 unique values
race_o: 5 unique values
importance_same_race: 17 unique values
importance_same_religion: 15 unique values
d_importance_same_race: 3 unique values
d_importance_same_religion: 3 unique values


In [178]:
df_dating["d_pref_o_attractive"].unique()

array(['[21-100]', '[16-20]', '[0-15]'], dtype=object)

In [179]:
df_dating["d_d_age"].unique()

array(['[4-6]', '[0-1]', '[2-3]', '[7-37]'], dtype=object)

In [180]:
df_dating["d_importance_same_religion"].unique()

array(['[2-5]', '[0-1]', '[6-10]'], dtype=object)

použit precision metric ---


In [181]:
df_dating.describe()

,has_null,wave,d_age,samerace,decision,decision_o,match
count,8378.00000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000
mean,0.87491,11.350919,4.185605,0.395799,0.419909,0.419551,0.164717
std,0.33084,5.995903,4.596171,0.489051,0.493573,0.493515,0.370947
min,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.00000,7.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.00000,11.000000,3.000000,0.000000,0.000000,0.000000,0.000000
75%,1.00000,15.000000,5.000000,1.000000,1.000000,1.000000,0.000000
max,1.00000,21.000000,37.000000,1.000000,1.000000,1.000000,1.000000


In [182]:
from sklearn.preprocessing import LabelEncoder

# Get all columns that contain range strings
range_cols = [col for col in df_dating.columns if col.startswith("d_")]

other_categorical = ["gender", "race", "race_o"]

cols_to_encode = range_cols + other_categorical

le = LabelEncoder()

for col in cols_to_encode:
    if col in df_dating.columns:
        if df_dating[col].dtype in ["object", "string"]:
            most_frequent = df_dating[col].mode()[0]
            df_dating[col] = df_dating[col].fillna(most_frequent)

            df_dating[col] = le.fit_transform(df_dating[col])

print("Encoded columns sample:")
print(df_dating[cols_to_encode].head())
print("\nData types:")
print(df_dating[cols_to_encode].dtypes)
print("\nUnique values in d_d_age:", df_dating["d_d_age"].unique())

Encoded columns sample:
   d_age  d_d_age  d_importance_same_race  d_importance_same_religion  \
0      6        2                       1                           1   
1      1        0                       1                           1   
2      1        0                       1                           1   
3      2        1                       1                           1   
4      3        1                       1                           1   

   d_pref_o_attractive  d_pref_o_sincere  d_pref_o_intelligence  \
0                    2                 1                      1   
1                    2                 0                      0   
2                    1                 1                      1   
3                    2                 0                      0   
4                    2                 0                      1   

   d_pref_o_funny  d_pref_o_ambitious  d_pref_o_shared_interests  \
0               1                   0                          0  

In [183]:
print("Total NaN values:", df_dating.isna().sum().sum())

print("\nNaN count per column:")
print(df_dating.isna().sum())

print("\nColumns with NaN:")
print(df_dating.isna().sum()[df_dating.isna().sum() > 0])

print("\nPercentage of NaN per column:")
print((df_dating.isna().sum() / len(df_dating)) * 100)

Total NaN values: 11658

NaN count per column:
has_null                0
wave                    0
gender                  0
age                    95
age_o                 104
                     ... 
d_guess_prob_liked      0
met                   375
decision                0
decision_o              0
match                   0
Length: 122, dtype: int64

Columns with NaN:
age                                95
age_o                             104
importance_same_race               79
importance_same_religion           79
field                              63
pref_o_attractive                  89
pref_o_sincere                     89
pref_o_intelligence                89
pref_o_funny                       98
pref_o_ambitious                  107
pref_o_shared_interests           129
attractive_o                      212
sinsere_o                         287
intelligence_o                    306
funny_o                           360
ambitous_o                        722
shared_interes

In [184]:
from sklearn.impute import SimpleImputer

# Separate numeric and categorical
numeric_cols = df_dating.select_dtypes(include=["float64", "int64"]).columns
categorical_cols = df_dating.select_dtypes(include=["object", "string"]).columns

# Impute numeric with median
imputer_num = SimpleImputer(strategy="median")
df_dating[numeric_cols] = imputer_num.fit_transform(df_dating[numeric_cols])

# Impute categorical with most frequent
imputer_cat = SimpleImputer(strategy="most_frequent")
if len(categorical_cols) > 0:
    df_dating[categorical_cols] = imputer_cat.fit_transform(df_dating[categorical_cols])


In [185]:
print("Total NaN values:", df_dating.isna().sum().sum())

print("\nNaN count per column:")
print(df_dating.isna().sum())

print("\nColumns with NaN:")
print(df_dating.isna().sum()[df_dating.isna().sum() > 0])

print("\nPercentage of NaN per column:")
print((df_dating.isna().sum() / len(df_dating)) * 100)

Total NaN values: 0

NaN count per column:
has_null              0
wave                  0
gender                0
age                   0
age_o                 0
                     ..
d_guess_prob_liked    0
met                   0
decision              0
decision_o            0
match                 0
Length: 122, dtype: int64

Columns with NaN:
Series([], dtype: int64)

Percentage of NaN per column:
has_null              0.0
wave                  0.0
gender                0.0
age                   0.0
age_o                 0.0
                     ... 
d_guess_prob_liked    0.0
met                   0.0
decision              0.0
decision_o            0.0
match                 0.0
Length: 122, dtype: float64


In [186]:
df_dating.head()

,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,importance_same_race,importance_same_religion,d_importance_same_race,d_importance_same_religion,field,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,d_pref_o_attractive,d_pref_o_sincere,d_pref_o_intelligence,d_pref_o_funny,d_pref_o_ambitious,d_pref_o_shared_interests,attractive_o,sinsere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,d_attractive_o,d_sinsere_o,d_intelligence_o,d_funny_o,d_ambitous_o,d_shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,d_attractive_important,d_sincere_important,d_intellicence_important,d_funny_important,d_ambtition_important,d_shared_interests_important,attractive,sincere,intelligence,funny,ambition,d_attractive,d_sincere,d_intelligence,d_funny,d_ambition,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,d_attractive_partner,d_sincere_partner,d_intelligence_partner,d_funny_partner,d_ambition_partner,d_shared_interests_partner,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,d_sports,d_tvsports,d_exercise,d_dining,d_museums,d_art,d_hiking,d_gaming,d_clubbing,d_reading,d_tv,d_theater,d_movies,d_concerts,d_music,d_shopping,d_yoga,interests_correlate,d_interests_correlate,expected_happy_with_sd_people,expected_num_matches,d_expected_happy_with_sd_people,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match
0,0.0,1.0,0.0,21,27,6.0,2.0,0.0,3.0,0.0,2,4,1.0,1.0,Law,35,20,20,20,0,5,2.0,1.0,1.0,1.0,0.0,0.0,6,8,8,8,8,6,1.0,1.0,1.0,1.0,1.0,1.0,15,20,20,15,15,15,0.0,1.0,1.0,0.0,0.0,0.0,6,8,8,8,7,1.0,1.0,1.0,1.0,1.0,6,9,7,7,6,5,1.0,2.0,1.0,1.0,1.0,0.0,9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,2.0,1.0,0.0,0.14,1.0,3,4,0.0,0.0,1.0,7,6,1.0,1.0,0,1.0,0.0,0.0
1,0.0,1.0,0.0,21,22,1.0,0.0,0.0,3.0,0.0,2,4,1.0,1.0,Law,60,0,0,40,0,0,2.0,0.0,0.0,2.0,0.0,0.0,7,8,10,7,7,5,1.0,1.0,2.0,1.0,1.0,0.0,15,20,20,15,15,15,0.0,1.0,1.0,0.0,0.0,0.0,6,8,8,8,7,1.0,1.0,1.0,1.0,1.0,7,8,7,8,5,6,1.0,1.0,1.0,1.0,0.0,1.0,9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,2.0,1.0,0.0,0.54,2.0,3,4,0.0,0.0,1.0,7,5,1.0,1.0,1,1.0,0.0,0.0
2,1.0,1.0,0.0,21,22,1.0,0.0,0.0,0.0,1.0,2,4,1.0,1.0,Law,19,18,19,18,14,12,1.0,1.0,1.0,1.0,0.0,0.0,10,10,10,10,10,10,2.0,2.0,2.0,2.0,2.0,2.0,15,20,20,15,15,15,0.0,1.0,1.0,0.0,0.0,0.0,6,8,8,8,7,1.0,1.0,1.0,1.0,1.0,5,8,9,8,5,7,0.0,1.0,2.0,1.0,0.0,1.0,9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,2.0,1.0,0.0,0.16,1.0,3,4,0.0,0.0,1.0,7,5,1.0,0.0,1,1.0,1.0,1.0
3,0.0,1.0,0.0,21,23,2.0,1.0,0.0,3.0,0.0,2,4,1.0,1.0,Law,30,5,15,40,5,5,2.0,0.0,0.0,2.0,0.0,0.0,7,8,9,8,9,8,1.0,1.0,2.0,1.0,2.0,1.0,15,20,20,15,15,15,0.0,1.0,1.0,0.0,0.0,0.0,6,8,8,8,7,1.0,1.0,1.0,1.0,1.0,7,6,8,7,6,8,1.0,1.0,1.0,1.0,1.0,1.0,9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,2.0,1.0,0.0,0.61,2.0,3,4,0.0,0.0,1.0,7,6,1.0,1.0,0,1.0,1.0,1.0
4,0.0,1.0,0.0,21,24,3.0,1.0,0.0,2.0,0.0,2,4,1.0,1.0,Law,30,10,20,10,10,20,2.0,0.0,1.0,0.0,0.0,1.0,8,7,9,6,9,7,1.0,1.0,2.0,1.0,2.0,1.0,15,20,20,15,15,15,0.0,1.0,1.0,0.0,0.0,0.0,6,8,8,8,7,1.0,1.0,1.0,1.0,1.0,5,6,7,7,6,6,0.0,1.0,1.0,1.0,1.0,1.0,9,2,8,9,1,1,5,1,5,6,9,1,10,10,9,8,1,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,2.0,1.0,0.0,0.21,1.0,3,4,0.0,0.0,1.0,6,6,1.0,1.0,0,1.0,1.0,1.0


In [187]:
df_dating["d_pref_o_attractive"].unique()

array([2., 1., 0.])

In [188]:
df_dating["d_d_age"].unique()

array([2., 0., 1., 3.])

In [189]:
df_dating["d_importance_same_religion"].unique()

array([1., 0., 2.])

# Precision matrix?
